In [1]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
import cv2
import os
import cvlib as cv
import numpy as np

In [2]:
classes=['man','woman']
model=load_model('gener_detection.model')
webcam = cv2.VideoCapture(0)

In [6]:
while webcam.isOpened():
    status, frame = webcam.read()
    face, confidence = cv.detect_face(frame)
    for idx,f in enumerate(face):
        (left_x,left_y,right_x,right_y)=f[0],f[1],f[2],f[3]
        cv2.rectangle(frame,(left_x,left_y),(right_x,right_y),(0,255,0),2)
        crop=np.copy(frame[left_y:right_y,left_x:right_x])
        if (crop.shape[0]<10) & (crop.shape[1]<10):
            continue
        crop=cv2.resize(crop,(96,96))
        crop=crop.astype("float")/255.0
        crop=img_to_array(crop)
        crop=np.expand_dims(crop,axis=0)
        conf=model.predict(crop)[0]
        idx=np.argmax(conf)
        label=classes[idx]
        label="{}:{:.2f}%".format(label,conf[idx]*100)
        if left_y-10 > 10:
            y=left_y-10
        else:
            y=left_y+10
        cv2.putText(frame,label,(left_x,y),cv2.FONT_HERSHEY_SIMPLEX,0.7,(0,255,0),2)
    cv2.imshow('gender_detection',frame)
        
    if cv2.waitKey(1) & 0xFF==ord('q'):
        break
webcam.release()
cv2.destroyAllWindows()